# Biostat M280 Homework 2

**Due May 11 @ 11:59PM**


## Q1. Nonnegative Matrix Factorization

Nonnegative matrix factorization (NNMF) was introduced by [Lee and Seung (1999)](https://www.nature.com/articles/44565) as an analog of principal components and vector quantization with applications in data compression and clustering. In this homework we consider algorithms for fitting NNMF and (optionally) high performance computing using graphical processing units (GPUs).

<img src="./nnmf.png" width="500" align="center"/>

In mathematical terms, one approximates a data matrix $\mathbf{X} \in \mathbb{R}^{m \times n}$ with nonnegative entries $x_{ij}$ by a product of two low-rank matrices $\mathbf{V} \in \mathbb{R}^{m \times r}$ and $\mathbf{W} \in \mathbb{R}^{r \times n}$ with nonnegative entries $v_{ik}$ and $w_{kj}$. Consider minimization of the squared Frobenius norm
$$
	L(\mathbf{V}, \mathbf{W}) = \|\mathbf{X} - \mathbf{V} \mathbf{W}\|_{\text{F}}^2 = \sum_i \sum_j \left(x_{ij} - \sum_k v_{ik} w_{kj} \right)^2, \quad v_{ik} \ge 0, w_{kj} \ge 0,
$$
which should lead to a good factorization. Later in the course we will learn how to derive a majorization-minimization (MM) algorithm with iterative updates
$$
	v_{ik}^{(t+1)} = v_{ik}^{(t)} \frac{\sum_j x_{ij} w_{kj}^{(t)}}{\sum_j b_{ij}^{(t)} w_{kj}^{(t)}}, \quad \text{where } b_{ij}^{(t)} = \sum_k v_{ik}^{(t)} w_{kj}^{(t)},
$$
$$
	w_{kj}^{(t+1)} = w_{kj}^{(t)} \frac{\sum_i x_{ij} v_{ik}^{(t+1)}}{\sum_i b_{ij}^{(t+1/2)} v_{ik}^{(t+1)}}, \quad \text{where } b_{ij}^{(t+1/2)} = \sum_k v_{ik}^{(t+1)} w_{kj}^{(t)}
$$
that drive the objective $L^{(t)} = L(\mathbf{V}^{(t)}, \mathbf{W}^{(t)})$ downhill. Superscript $t$ indicates iteration number. Efficiency (both speed and memory) will be the most important criterion when grading this problem.


1. Implement the algorithm with arguments: $\mathbf{X}$ (data, each row is a vectorized image), rank $r$, convergence tolerance, and optional starting point.
```julia
function nnmf(
    X::Matrix{T},
    r::Integer;
    maxiter::Integer=1000, 
    tol::Number=1e-4,
    V::Matrix{T}=rand(T, size(X, 1), r),
    W::Matrix{T}=rand(T, r, size(X, 2))
    ) where T <: AbstractFloat
    # implementation
    # Output
    return V, W
end
```

0. Database 1 from the [MIT Center for Biological and Computational Learning (CBCL)](http://cbcl.mit.edu) reduces to a matrix $\mathbf{X}$ containing $m = 2,429$ gray-scale face images with $n = 19 \times 19 = 361$ pixels per face. Each image (row) is scaled to have mean and standard deviation 0.25.  
Read in the [`nnmf-2429-by-361-face.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt) file, e.g., using [`readdlm()`](https://docs.julialang.org/en/stable/stdlib/io-network/#Base.DataFmt.readdlm-Tuple{Any,Char,Type,Char}) function, and display a couple sample images, e.g., using [ImageView.jl](https://github.com/JuliaImages/ImageView.jl) package.

0. Report the run times, using `@time`, of your function for fitting NNMF on the MIT CBCL face data set at ranks $r=10, 20, 30, 40, 50$. For ease of comparison (and grading), please start your algorithm with the provided $\mathbf{V}^{(0)}$ (first $r$ columns of [`V0.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/V0.txt)) and $\mathbf{W}^{(0)}$ (first $r$ rows of [`W0.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/W0.txt)) and stopping criterion
$$
	\frac{|L^{(t+1)} - L^{(t)}|}{|L^{(t)}| + 1} \le 10^{-4}.
$$

0. Choose an $r \in \{10, 20, 30, 40, 50\}$ and start your algorithm from a different $\mathbf{V}^{(0)}$ and $\mathbf{W}^{(0)}$. Do you obtain the same objective value and $(\mathbf{V}, \mathbf{W})$? Explain what you find.

0. For the same $r$, start your algorithm from $v_{ik}^{(0)} = w_{kj}^{(0)} = 1$ for all $i,j,k$. Do you obtain the same objective value and $(\mathbf{V}, \mathbf{W})$? Explain what you find.

0. Plot the basis images (rows of $\mathbf{W}$) at rank $r=50$. What do you find?

0. (Optional) Investigate the GPU capabilities of Julia. Report the speed gain of your GPU code over CPU code at ranks $r=10, 20, 30, 40, 50$. Make sure to use the same starting point as in part 2.

## 1. Implement the algorithm with arguments:  X

In [36]:
function mynnmf(X::Matrix{T}, r::Int, 
        tol::Float64 = 1e-4, maxiter = 100,
        V::Matrix{T}=rand(T, size(X, 1), r),
        W::Matrix{T}=rand(T, r, size(X, 2)),
    ) where T <: AbstractFloat
    
    m = size(X, 1)
    n = size(X, 2)
    mn = m * n
    loss = 0
    # Iteration
    for iter = 1: maxiter    
        for k = 1:r
            for i = 1:m
                num = 0
                den = 0
                for j = 1:n 
                    num = num + X[i, j]*W[k, j]
                    b = 0
                    for t = 1: r
                        b = b + V[i, t]*W[t, j]
                    end
                    den = den + b*W[k, j]
                        
                end 
                V[i, k] = V[i, k]*num/den
            end
        end
        for j = 1:n  
            for k = 1:r
                num1 = 0
                den1 = 0
                for i = 1:m
                    num1 = num1 +  X[i, j]*V[i, k]
                    b1 = 0
                    for t = 1:r
                        b1 = b1 + V[i, t]*W[t, j]
                    end
                    den1 = den1 + b1*V[i, k]                   
                end
                W[k, j] = W[k, j]*num1/den1
            end
        end
        # minimization of the squared Frobenius norm
        B = V * W
        diff = X - B
        lastloss = loss
        loss = vecnorm(diff)^2
         # convergency or not
        if abs(loss - lastloss) < tol
            break
        end
    end   
    #return
    return V, W
end

mynnmf (generic function with 5 methods)

In [37]:
#try nnmf

X = rand(10,10)
r = 5 
mynnmf(X, r)



([0.378516 0.374429 … 0.421753 0.000273606; 1.25823e-16 0.487918 … 1.27412e-8 0.196669; … ; 0.191274 1.10364e-6 … 0.193434 0.646785; 1.50366e-5 0.0262135 … 0.069454 0.228487], [1.33988 0.590325 … 0.715795 0.00108701; 0.248739 0.0190636 … 0.178541 0.00324965; … ; 0.138944 0.707938 … 0.0989725 0.643369; 0.0130991 0.141086 … 0.337713 1.0727])

In [38]:
# readin matrix X
path_X = "http://Hua-Zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt"
X = readdlm(download(path_X), ' ')

# readin matrix V0
path_V0 = "http://Hua-Zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/V0.txt"
V0 = readdlm(download(path_V0), ' ')

# readin matrix W0
path_W0 = "http://Hua-Zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/W0.txt"
W0 = readdlm(download(path_W0), ' ');

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6012k  100 6012k    0     0  3871k      0  0:00:01  0:00:01 --:--:-- 3873k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  948k  100  948k    0     0  1615k      0 --:--:-- --:--:-- --:--:-- 1613k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  140k  100  140k    0     0   989k      0 --:--:-- --:--:-- --:--:--  992k


In [41]:
Pkg.add("ImageView")

INFO: Cloning cache of AbstractFFTs from https://github.com/JuliaMath/AbstractFFTs.jl.git
INFO: Cloning cache of AxisAlgorithms from https://github.com/timholy/AxisAlgorithms.jl.git
INFO: Cloning cache of Cairo from https://github.com/JuliaGraphics/Cairo.jl.git
INFO: Cloning cache of CatIndices from https://github.com/JuliaArrays/CatIndices.jl.git
INFO: Cloning cache of ColorVectorSpace from https://github.com/JuliaGraphics/ColorVectorSpace.jl.git
INFO: Cloning cache of ComputationalResources from https://github.com/timholy/ComputationalResources.jl.git
INFO: Cloning cache of CoordinateTransformations from https://github.com/FugroRoames/CoordinateTransformations.jl.git
INFO: Cloning cache of CustomUnitRanges from https://github.com/JuliaArrays/CustomUnitRanges.jl.git
INFO: Cloning cache of Distances from https://github.com/JuliaStats/Distances.jl.git
INFO: Cloning cache of DualNumbers from https://github.com/JuliaDiff/DualNumbers.jl.git
INFO: Cloning cache of FFTViews from https://gith

Updated 1 tap (homebrew/core).
==> New Formulae
cglm
click
deployer
gcc@7
genact
graph-tool
hapi-fhir-cli
libbitcoin-client
libde265
libplctag
petsc
stm32flash
unp64
woff2
xidel
yamllint
==> Updated Formulae
abcm2ps
abcmidi
abyss
acme
acpica
activemq
adplug
aircrack-ng
akamai
algernon
amqp-cpp
angular-cli
annie
ansible
ansible-cmdb
apache-geode
apktool
arangodb
archivemount
armadillo
armor
arpack
artifactory
asciidoctor
atari800
atdtool
aubio
audacious
autossh
avimetaedit
aws-sdk-cpp
awscli
azure-cli
baresip
bartycrouch
basex
bash-completion@2
bazel
bedops
bento4
bettercap
binaryen
binwalk
bit
bitrise
blockhash
bluepill
bochs
bower
burp
bzt
cake
cayley
certbot
cfitsio
cfr-decompiler
cgal
cgrep
chakra
chamber
chapel
check_postgres
checkbashisms
checkstyle
chronograf
cjdns
ckan
clojurescript
cockroach
coffeescript
composer
conan
convox
coq
corsixth
cp2k
cpanminus
ctl
dar
dash
datetime-fortran
dbus
dcd
deark
dehydrated
derby
dhall-json
diffoscope
dislocker
django-completion
dmd
dmtx-utils

INFO: Building Cairo


==> Installing gettext from staticfloat/juliatranslated
==> Downloading https://homebrew.bintray.com/bottles/gettext-0.19.8.1.sierra.bottle.tar.gz
==> Pouring gettext-0.19.8.1.sierra.bottle.tar.gz
==> Caveats
This formula is keg-only, which means it was not symlinked into /Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr,
because macOS provides the BSD gettext library & some software gets confused if both are in the library path.

If you need to have this software first in your PATH run:
  echo 'export PATH="/Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/opt/gettext/bin:$PATH"' >> ~/.bash_profile

For compilers to find this software you may need to set:
    LDFLAGS:  -L/Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/opt/gettext/lib
    CPPFLAGS: -I/Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/opt/gettext/include

==> Summary
🍺  /Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/Cellar/gettext/0.19.8.1: 1,934 files, 16.9MB
==> Installing pcre from staticfloat/juliatranslated
==> Downloading https://home

==> Installing libffi from staticfloat/juliatranslated
==> Downloading https://homebrew.bintray.com/bottles/libffi-3.2.1.sierra.bottle.tar.gz
==> Pouring libffi-3.2.1.sierra.bottle.tar.gz
==> Caveats
This formula is keg-only, which means it was not symlinked into /Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr,
because some formulae require a newer version of libffi.

For compilers to find this software you may need to set:
    LDFLAGS:  -L/Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/opt/libffi/lib

==> Summary
🍺  /Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/Cellar/libffi/3.2.1: 16 files, 297KB
==> Installing glib from staticfloat/juliatranslated
==> Downloading https://homebrew.bintray.com/bottles/glib-2.56.1.sierra.bottle.tar.gz
==> Pouring glib-2.56.1.sierra.bottle.tar.gz
🍺  /Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/Cellar/glib/2.56.1: 430 files, 23.7MB


...

==> Installing gsettings-desktop-schemas from staticfloat/juliatranslated
==> Downloading https://homebrew.bintray.com/bottles/gsettings-desktop-schemas-3.28.0.sierra.bottle.tar.gz
==> Pouring gsettings-desktop-schemas-3.28.0.sierra.bottle.tar.gz
==> /Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/opt/glib/bin/glib-compile-schemas /Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/share/glib-2.0/schemas
🍺  /Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/Cellar/gsettings-desktop-schemas/3.28.0: 96 files, 4.0MB


==> Installing libepoxy from staticfloat/juliatranslated
==> Downloading https://homebrew.bintray.com/bottles/libepoxy-1.5.0_1.sierra.bottle.tar.gz
==> Pouring libepoxy-1.5.0_1.sierra.bottle.tar.gz
🍺  /Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/Cellar/libepoxy/1.5.0_1: 11 files, 2.6MB


==> Installing atk from staticfloat/juliatranslated
==> Downloading https://homebrew.bintray.com/bottles/atk-2.28.1_2.sierra.bottle.tar.gz
==> Pouring atk-2.28.1_2.sierra.bottle.tar.gz
🍺  /Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/Cellar/atk/2.28.1_2: 144 files, 2.2MB


==> Installing gtk+3 from staticfloat/juliatranslated
==> Downloading https://homebrew.bintray.com/bottles/gtk+3-3.22.30.sierra.bottle.tar.gz
==> Pouring gtk+3-3.22.30.sierra.bottle.tar.gz
==> /Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/opt/glib/bin/glib-compile-schemas /Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/share/glib-2.0/schemas
🍺  /Users/huiyuhu/.julia/v0.6/Homebrew/deps/usr/Cellar/gtk+3/3.22.30: 1,373 files, 68.7MB


INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of ImageView
INFO: Use `Pkg.update()` to get the latest versions of your packages


## Q2. Linear Mixed Models

Consider a linear mixed effects model
$$
	y_i = \mathbf{x}_i^T \beta + \mathbf{z}_i^T \gamma + \epsilon_i, \quad i=1,\ldots,n,
$$
where $\epsilon_i$ are independent normal errors $N(0,\sigma_0^2)$, $\beta \in \mathbb{R}^p$ are fixed effects, and $\gamma \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \sigma_1^2 \mathbf{I}_q$) independent of $\epsilon_i$. 

0. Show that 
$$
    \mathbf{y} \sim N \left( \mathbf{X} \beta, \sigma_0^2 \mathbf{I}_n + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T \right),
$$
where $\mathbf{y} = (y_1, \ldots, y_n)^T \in \mathbb{R}^n$, $\mathbf{X} = (\mathbf{x}_1, \ldots, \mathbf{x}_n)^T \in \mathbb{R}^{n \times p}$, and $\mathbf{Z} = (\mathbf{z}_1, \ldots, \mathbf{z}_n)^T \in \mathbb{R}^{n \times q}$. 

0. Write a function, with interface 
    ```julia
    logpdf_mvn(y::Vector, Z::Matrix, σ0::Number, σ1::Number),
    ```
that evaluates the log-density of a multivariate normal with mean $\mathbf{0}$ and covariance $\sigma_0^2 \mathbf{I} + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T$ at $\mathbf{y}$. Make your code efficient in the $n \gg q$ case. 

0. Compare your result (both accuracy and timing) to the [Distributions.jl](http://distributionsjl.readthedocs.io/en/latest/multivariate.html#multivariate-normal-distribution) package using following data.  
    ```julia
    using BenchmarkTools, Distributions

    srand(280)
    n, q = 2000, 10
    Z = randn(n, q)
    σ0, σ1 = 0.5, 2.0
    Σ = σ1^2 * Z * Z.' + σ0^2 * I
    mvn = MvNormal(Σ) # MVN(0, Σ)
    y = rand(mvn) # generate one instance from MNV(0, Σ)

    # check you answer matches that from Distributions.jl
    @show logpdf_mvn(y, Z, σ0, σ1)
    @show logpdf(mvn, y)

    # benchmark
    @benchmark logpdf_mvn(y, Z, σ0, σ1)
    @benchmark logpdf(mvn, y)
    ```